<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [26]:
#Importing libraries
import pandas as pd
import numpy as np
from graphviz import Digraph
from matplotlib import pyplot as plt
import datetime

In [27]:
df = pd.read_csv("Helpdesk.csv")

In [28]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [30]:
df.drop_duplicates(inplace=True)

In [31]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='ISO8601')

<ipython-input-31-97bac535b9d0>:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='ISO8601')


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21221 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21221 non-null  object
 1   lifecycle:transition  21221 non-null  object
 2   org:resource          21221 non-null  object
 3   time:timestamp        21221 non-null  object
 4   Activity              21221 non-null  object
 5   Resource              21221 non-null  object
 6   case:concept:name     21221 non-null  object
 7   case:variant          21221 non-null  object
 8   case:variant-index    21221 non-null  int64 
 9   case:creator          21221 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.8+ MB


In [33]:
df['new_date'] = [d.date() for d in df['time:timestamp']]
df['new_time'] = [d.time() for d in df['time:timestamp']]

In [34]:
df['time_lapsed'] = df.groupby('case:concept:name')['time:timestamp'].transform('min')

In [36]:
df['time_lapsed'] = df['time:timestamp'] - df['time_lapsed']

In [37]:
df['concept:name'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Wait', 'Create SW anomaly', 'Insert ticket',
       'Schedule intervention', 'RESOLVED', 'INVALID', 'VERIFIED',
       'Resolve SW anomaly', 'Require upgrade', 'DUPLICATE'], dtype=object)

In [38]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [39]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([ True])

In [40]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2','case:creator','time:timestamp','lifecycle:transition'],axis=1, inplace=True)

In [41]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#This is also to immulate the label encoder
#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [42]:
df=pd.merge(df,stage,on='Activity', how='left')

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21221 entries, 0 to 21220
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Activity            21221 non-null  object 
 1   Resource            21221 non-null  object 
 2   case:concept:name   21221 non-null  object 
 3   case:variant        21221 non-null  object 
 4   case:variant-index  21221 non-null  int64  
 5   new_date            21221 non-null  object 
 6   new_time            21221 non-null  object 
 7   time_lapsed         21221 non-null  object 
 8   Stage_no            21142 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 1.5+ MB


In [44]:
df.head()

,Activity,Resource,case:concept:name,case:variant,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,Assign seriousness,Value 2,Case3608,Variant 33,33,2010-01-13,08:40:25,0 days 00:00:00,2.0
1,Take in charge ticket,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:27,16 days 00:12:02,3.0
2,Resolve ticket,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:34,16 days 00:12:09,7.0
3,Closed,Value 5,Case3608,Variant 33,33,2010-02-13,08:52:48,31 days 00:12:23,10.0
4,Assign seriousness,Value 2,Case2748,Variant 1,1,2010-01-13,12:26:04,0 days 00:00:00,2.0


One-hot-Encoding

In [123]:
from sklearn.preprocessing import OneHotEncoder

In [124]:
encoder = OneHotEncoder(sparse_output=False)

In [125]:
activities = df[['Activity']]
one_hot_encoded = encoder.fit_transform(activities)

In [126]:
encoded_df = pd.DataFrame(
    one_hot_encoded,
    columns=encoder.get_feature_names_out(['Activity'])
)

In [127]:
df = pd.concat([df.drop('Activity', axis=1), encoded_df], axis=1)

In [128]:
df.head()

,Resource,case:concept:name,case:variant,case:variant-index,new_date,new_time,Stage_no,Activity_Assign seriousness,Activity_Closed,Activity_Create SW anomaly,...,Activity_INVALID,Activity_Insert ticket,Activity_RESOLVED,Activity_Require upgrade,Activity_Resolve SW anomaly,Activity_Resolve ticket,Activity_Schedule intervention,Activity_Take in charge ticket,Activity_VERIFIED,Activity_Wait
0,Value 2,Case3608,Variant 33,33,2010-01-13,08:40:25,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:27,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Value 2,Case3608,Variant 33,33,2010-01-29,08:52:34,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Value 5,Case3608,Variant 33,33,2010-02-13,08:52:48,10.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Value 2,Case2748,Variant 1,1,2010-01-13,12:26:04,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
df = df.sort_values(['new_date', 'new_time'])

In [131]:
df.head()

,Resource,case:concept:name,case:variant,case:variant-index,new_date,new_time,Stage_no,Activity_Assign seriousness,Activity_Closed,Activity_Create SW anomaly,...,Activity_INVALID,Activity_Insert ticket,Activity_RESOLVED,Activity_Require upgrade,Activity_Resolve SW anomaly,Activity_Resolve ticket,Activity_Schedule intervention,Activity_Take in charge ticket,Activity_VERIFIED,Activity_Wait
0,Value 2,Case3608,Variant 33,33,2010-01-13,08:40:25,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Value 2,Case2748,Variant 1,1,2010-01-13,12:26:04,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Value 2,Case4284,Variant 219,219,2010-01-13,12:30:37,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,Value 2,Case1534,Variant 33,33,2010-01-13,13:09:31,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,Value 2,Case406,Variant 4,4,2010-01-13,17:25:25,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Model building

In [84]:
!pip install keras

In [86]:
#Creating the model architecture

In [85]:
#Importing the required libriares since the LSTM model will be done with Pytorch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization